# Competition

# Competition- [Tabular Playground Series - Jan 2021](https://www.kaggle.com/c/tabular-playground-series-jan-2021/overview)

### Approach(TOP-19%)
* I have use Models - Extreme Gradient Boosting,Light Gradient Boosting and Catboost Regressor.
* There is only early stopping criteria is met with no other hyperparameter tuning, and 5 fold Cross-validation is used for making model robust.
* My first approach(Ensembeling) is submitting average prediction of all these models- Out of fold cross validation score-0.6982 , Public leaderboard Score-0.69971
* Second approach(Stacking) I have added output of these 3 models as 3 more features in the model and train Light Gradient Boosting model, Out of fold cross validation score-0.6978 , Public leaderboard Score-0.69908

In [ ]:
# import Libraries
import pandas as pd
import numpy as np
import matplotlib as pyplot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# to see all the comands result in a single kernal 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to increase no. of rows and column visibility in outputs
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)

#To ignore warnings
import warnings
warnings.simplefilter('ignore')

In [ ]:
#Import data
train = pd.read_csv(r'../input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv(r'../input/tabular-playground-series-jan-2021/test.csv')
sample = pd.read_csv(r'../input/tabular-playground-series-jan-2021/sample_submission.csv')

# Basic Visualization

In [ ]:
# Having a look at data and its shape 
train.head()
test.head()
train.shape ,test.shape ,sample.shape

In [ ]:
# Missing value check 
train.isna().sum().sum()
test.isna().sum().sum()

In [ ]:
# Analysing Distribution of Variables
train.describe().T
test.describe().T

In [ ]:
#Target Distribution check
train['target'].describe([0,0.01,0.03,0.05,0.1,0.25,0.5,0.75,0.95,0.97,0.99,1])

In [ ]:
# found out that there is no. missing value and only one address as object type variable
# Target varibale distribution 
train['target'].plot(kind = 'density', title = 'target Distribution')

In [ ]:
# Checking correlation
plt.figure(figsize=(15, 8))
sns.heatmap(train.corr(),annot=True)

# Modeling

In [ ]:
#Importing Packages
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV,KFold
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
# Code for cross validation
def run_gradient_boosting(clf,k, fit_params, train, test, features):
  N_SPLITS = k
  oofs = np.zeros(len(train))
  preds = np.zeros((len(test)))

  folds = KFold(n_splits = N_SPLITS,random_state=2021)

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['target'])):
    print(f'\n------------- Fold {fold_ + 1} -------------')
    target=train['target']
    X_trn, y_trn = train[features].iloc[trn_idx], target.iloc[trn_idx]

    X_val, y_val = train[features].iloc[val_idx], target.iloc[val_idx]

    X_test = test[features]
    
    _ = clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], **fit_params)
    preds_val = clf.predict(X_val)
    preds_test = clf.predict(X_test)
    fold_score =  np.sqrt(mean_squared_error(y_val,preds_val)) 
    print(f'\n RMLSE score for validation set is {fold_score}')
    oofs[val_idx] = ((preds_val))
    preds += preds_test / N_SPLITS


  oofs_score = np.sqrt(mean_squared_error(target, oofs))
  print(f'\n\n rmlse score for oofs is {oofs_score}')

  return oofs, preds


In [ ]:
train.columns

In [ ]:
model_col=['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']
model_col_1=['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14','xgb','lgb','cb']

In [ ]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold

xgb = XGBRegressor(n_estimators = 8000,
#                        learning_rate = 0.05,
#                       colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'target'
features =model_col
xgb_oofs, xgb_preds = run_gradient_boosting(xgb,5, fit_params, train, test, features)

In [ ]:
lgb = LGBMRegressor(n_estimators=5000, importance_type='gain',
#                          learning_rate = 0.05,
#                          colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'target'
features =model_col
lgb_oofs, lgb_preds = run_gradient_boosting(lgb,5, fit_params, train, test, features)

In [ ]:
from catboost import CatBoostRegressor
cb = CatBoostRegressor(iterations=10000,
                       learning_rate = 0.1,
#                         colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False , 'early_stopping_rounds': 100}
TARGET_COL= 'target'
features =model_col
cb_oofs, cb_preds = run_gradient_boosting(cb,5, fit_params, train, test, features)

In [ ]:
# Feature Importance 
# you can also change cb to xgb or lgb to see their imporatance of variable
feat_importances = pd.Series(cb.feature_importances_, index=model_col)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

In [ ]:
train['xgb']=(xgb_oofs) 
train['lgb']=(lgb_oofs)
train['cb']=(cb_oofs)
test['xgb']=(xgb_preds)
test['lgb']=(lgb_preds)
test['cb']=(cb_preds)

In [ ]:
np.sqrt(mean_squared_error(train['target'],(xgb_oofs)))
np.sqrt(mean_squared_error(train['target'],(lgb_oofs)))
np.sqrt(mean_squared_error(train['target'],(cb_oofs)))
t=0.33*train['xgb']+0.33*train['lgb']+0.34*train['cb']
np.sqrt(mean_squared_error(train['target'],t))
test['target']=0.33*test['xgb']+0.33*test['lgb']+0.34*test['cb']

In [ ]:
output = pd.DataFrame({"id":test.id, "target":test.target})
output.to_csv('Ensembeling.csv', index=False)

In [ ]:
lgb2 = LGBMRegressor(n_estimators=5000, importance_type='gain',
#                          learning_rate = 0.05,
#                          colsample_bytree = 0.76,
                        )
fit_params = {'verbose': False, 'early_stopping_rounds': 100}
TARGET_COL= 'target'
features =model_col_1
lgb2_oofs, lgb2_preds = run_gradient_boosting(lgb2,5, fit_params, train, test, features)

In [ ]:
train['lgb2']=(lgb2_oofs)
test['lgb2']=(lgb2_preds)
output = pd.DataFrame({"id":test.id, "target":test.target})
output.to_csv('Stacking.csv', index=False)